# __CNN__

We decide to use CNN(convolutional neural networks) for the task of guitar tablature estimation. The previous work of Andrew Wiggins and Youngmoo Kim showed that CNNs have shown promise for translating guitar audios to tabs, and the use of CNNs has also been explored for various other tasks within music information retrieval such as musical tempo estimation, key classification, singing voice detection, and instrument classification. It is proven that CNN is a powerful tool for the purpose of our study.

## __Import libraries__

In [12]:
# Import required packages 
import tensorflow as tf
import tensorflow_hub as hub
import datetime
import pathlib
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from keras import backend as K
from PIL import Image

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
#Clear any logs from previous runs
!rm -rf ./logs/

In [6]:
# Check for Tensorflow version
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

2.8.0


## __"Write Python Script" function__

`%%write_and_run image_modeling.py` is the call of the register cell magic from below in 'w' mode (default). It writes the imports at the beginning of the `image_modeling.py`.

In [ ]:
'''# Let's make some dark cell magic. Why not!

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
        print("Appended to file ", file)
    else:
        print('Written to file:', file)
    with open(file, mode) as f:
        f.write(cell.format(**globals()))        
    get_ipython().run_cell(cell)'''

## __Define Input Shapes__

In [ ]:
#Import variables from our file
Frame_height = 192
FRAME_WIDTH = 9
N_CLASSES = 21
N_STRINGS = 6
BATCH_SIZE = 128
epochs = 8
key = "c"
data_path=
annotation_file = "annotation"
save_path=
TRAIN_PATH = 'flowers_train.csv'
EVAL_PATH = 'flowers_eval.csv'
TEST_PATH = 'flowers_test.csv'

TRAINING_SIZE = !wc -l < flowers_train.csv
TRAINING_STEPS = int(TRAINING_SIZE[0]) // BATCH_SIZE



## __Load Annotation__

In [ ]:

def load_IDs():
        
    save_folder = save_path + key + " " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "/"
    if not os.path.exists(save_folder):
            os.makedirs(save_folder)
    log_file = save_folder + "log.txt"

    #log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    metrics = {}
    metrics["pp"] = []
    metrics["pr"] = []
    metrics["pf"] = []
    metrics["tp"] = []
    metrics["tr"] = []
    metrics["tf"] = []
    metrics["tdr"] = []
    metrics["data"] = ["g0","g1","g2","g3","g4","g5","mean","std dev"]

    if key == "c":
        input_shape = (192, FRAME_WIDTH, 1)
    elif key == "m":
        input_shape = (128, FRAME_WIDTH, 1)
    elif key == "cm":
        input_shape = (320, FRAME_WIDTH, 1)
    elif key == "s":
        input_shape = (1025, FRAME_WIDTH, 1)
    

In [ ]:
def load_IDs():
        csv_file = data_path + annotation_file
        list_IDs = list(pd.read_csv(csv_file, header=None)[0])
        return list_IDs

## __Do train & test split__

In [ ]:
def partition_data(data_split):
    #we have do to an easier way of splitting data
        '''data_split = data_split 
        partition = {}
        partition["training"] = []
        partition["validation"] = []
        for ID in list_IDs:
            guitarist = int(ID.split("_")[0])
            if guitarist == data_split:
                partition["validation"].append(ID)
            else:
                partition["training"].append(ID)
                
        training_generator = DataGenerator(partition['training'], 
                                                data_path=data_path, 
                                                batch_size=batch_size, 
                                                shuffle=True,
                                                spec_repr=key, 
                                                con_win_size= FRAME_WIDTH)
        
        validation_generator = DataGenerator(partition['validation'], 
                                                data_path=data_path, 
                                                batch_size=len(partition['validation']), 
                                                shuffle=False,
                                                spec_repr=spec_repr, 
                                                con_win_size=con_win_size)'''
        
        split_folder = save_folder + str(data_split) + "/"
        if not os.path.exists(split_folder):
            os.makedirs(split_folder)

## __Define functions__

In [ ]:
def log_model():
        with open(log_file,'w') as fh:
            fh.write("\nbatch_size: " + str(BATCH_SIZE))
            fh.write("\nepochs: " + str(epochs))
            fh.write("\nkey: " + str(key))
            fh.write("\ndata_path: " + str(data_path))
            fh.write("\nframe_width: " + str(FRAME_WIDTH))
            fh.write("\nannotation_file: " + str(annotation_file) + "\n")
            swizzle_model.summary(print_fn=lambda x: fh.write(x + '\n'))
       
    def softmax_by_string(t):
        sh = K.shape(t)
        string_sm = []
        for i in range(N_STRINGS):
            string_sm.append(K.expand_dims(K.softmax(t[:,i,:]), axis=1))
        return K.concatenate(string_sm, axis=1)
    
    def catcross_by_string(target, output):
        loss = 0
        for i in range(N_STRINGS):
            loss += K.categorical_crossentropy(target[:,i,:], output[:,i,:])
        return loss
    
    def avg_acc(y_true, y_pred):
        return K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))

Using Tensorboard to monitor our results:

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# Write a testing function.
def test(model):
    
    test_dataset = image_modeling.load_dataset(TEST_PATH, batch_size=1, training=False)
    model.evaluate(test_dataset)

# Call the testing function for our model
test(model)

## Building our CNN Model

In [ ]:
def cnn_swizzle_model():       
        swizzle_model = tf.keras.swizzle_models.Sequential()
        swizzle_model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3),activation='relu',input_shape=self.input_shape))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        swizzle_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        swizzle_model.add(tf.keras.layers.Dropout(0.25))   
        swizzle_model.add(tf.keras.layers.Flatten())
        swizzle_model.add(tf.keras.layers.Dense(128, activation='relu'))
        swizzle_model.add(tf.keras.layers.Dropout(0.5))
        swizzle_model.add(tf.keras.layers.Dense(self.num_classes * self.num_strings)) # no activation
        swizzle_model.add(tf.keras.layers.Reshape((self.num_strings, self.num_classes)))
        swizzle_model.add(tf.keras.layers.Activation(self.softmax_by_string))
        swizzle_model = swizzle_model

## __Train CNN__

In [ ]:
def train():
        swizzle_model.fit_generator(generator=training_generator,
                    validation_data=None,
                    epochs=epochs,
                    verbose=1,
                    use_multiprocessing=True,
                    workers=9)

## Transfer Learning

We want to use Transfer learning where a model is trained on one task and is then reused for another task. One approach to transfer learning is fine-tuning.

## __Tab/CNN__

In [ ]:
''' A CNN to classify 6 fret-string positions
    at the frame level during guitar performance
'''

from __future__ import print_function
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D, Lambda
from keras import backend as K
from DataGenerator import DataGenerator
import pandas as pd
import numpy as np
import datetime
from Metrics import *

class TabCNN:
    
    def __init__(self, 
                 batch_size=128, 
                 epochs=8,
                 con_win_size = 9,
                 spec_repr="c",
                 data_path="../data/spec_repr/",
                 id_file="id.csv",
                 save_path="saved/"):   
        
        self.batch_size = batch_size
        self.epochs = epochs
        self.con_win_size = con_win_size
        self.spec_repr = spec_repr
        self.data_path = data_path
        self.id_file = id_file
        self.save_path = save_path
        
        self.load_IDs()
        
        self.save_folder = self.save_path + self.spec_repr + " " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "/"
        if not os.path.exists(self.save_folder):
            os.makedirs(self.save_folder)
        self.log_file = self.save_folder + "log.txt"
        
        self.metrics = {}
        self.metrics["pp"] = []
        self.metrics["pr"] = []
        self.metrics["pf"] = []
        self.metrics["tp"] = []
        self.metrics["tr"] = []
        self.metrics["tf"] = []
        self.metrics["tdr"] = []
        self.metrics["data"] = ["g0","g1","g2","g3","g4","g5","mean","std dev"]
        
        if self.spec_repr == "c":
            self.input_shape = (192, self.con_win_size, 1)
        elif self.spec_repr == "m":
            self.input_shape = (128, self.con_win_size, 1)
        elif self.spec_repr == "cm":
            self.input_shape = (320, self.con_win_size, 1)
        elif self.spec_repr == "s":
            self.input_shape = (1025, self.con_win_size, 1)
            
        # these probably won't ever change
        self.num_classes = 21
        self.num_strings = 6

    def load_IDs(self):
        csv_file = self.data_path + self.id_file
        self.list_IDs = list(pd.read_csv(csv_file, header=None)[0])
        
    def partition_data(self, data_split):
        self.data_split = data_split
        self.partition = {}
        self.partition["training"] = []
        self.partition["validation"] = []
        for ID in self.list_IDs:
            guitarist = int(ID.split("_")[0])
            if guitarist == data_split:
                self.partition["validation"].append(ID)
            else:
                self.partition["training"].append(ID)
                
        self.training_generator = DataGenerator(self.partition['training'], 
                                                data_path=self.data_path, 
                                                batch_size=self.batch_size, 
                                                shuffle=True,
                                                spec_repr=self.spec_repr, 
                                                con_win_size=self.con_win_size)
        
        self.validation_generator = DataGenerator(self.partition['validation'], 
                                                data_path=self.data_path, 
                                                batch_size=len(self.partition['validation']), 
                                                shuffle=False,
                                                spec_repr=self.spec_repr, 
                                                con_win_size=self.con_win_size)
        
        self.split_folder = self.save_folder + str(self.data_split) + "/"
        if not os.path.exists(self.split_folder):
            os.makedirs(self.split_folder)
                
    def log_model(self):
        with open(self.log_file,'w') as fh:
            fh.write("\nbatch_size: " + str(self.batch_size))
            fh.write("\nepochs: " + str(self.epochs))
            fh.write("\nspec_repr: " + str(self.spec_repr))
            fh.write("\ndata_path: " + str(self.data_path))
            fh.write("\ncon_win_size: " + str(self.con_win_size))
            fh.write("\nid_file: " + str(self.id_file) + "\n")
            self.model.summary(print_fn=lambda x: fh.write(x + '\n'))
       
    def softmax_by_string(self, t):
        sh = K.shape(t)
        string_sm = []
        for i in range(self.num_strings):
            string_sm.append(K.expand_dims(K.softmax(t[:,i,:]), axis=1))
        return K.concatenate(string_sm, axis=1)
    
    def catcross_by_string(self, target, output):
        loss = 0
        for i in range(self.num_strings):
            loss += K.categorical_crossentropy(target[:,i,:], output[:,i,:])
        return loss
    
    def avg_acc(self, y_true, y_pred):
        return K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))
           
    def build_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                             activation='relu',
                             input_shape=self.input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))   
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes * self.num_strings)) # no activation
        model.add(Reshape((self.num_strings, self.num_classes)))
        model.add(Activation(self.softmax_by_string))

        model.compile(loss=self.catcross_by_string,
                      optimizer=keras.optimizers.Adadelta(),
                      metrics=[self.avg_acc])
        
        self.model = model

    def train(self):
        self.model.fit_generator(generator=self.training_generator,
                    validation_data=None,
                    epochs=self.epochs,
                    verbose=1,
                    use_multiprocessing=True,
                    workers=9)
        
    def save_weights(self):
        self.model.save_weights(self.split_folder + "weights.h5")
        
    def test(self):
        self.X_test, self.y_gt = self.validation_generator[0]
        self.y_pred = self.model.predict(self.X_test)
        
    def save_predictions(self):
        np.savez(self.split_folder + "predictions.npz", y_pred=self.y_pred, y_gt=self.y_gt)
        
    def evaluate(self):
        self.metrics["pp"].append(pitch_precision(self.y_pred, self.y_gt))
        self.metrics["pr"].append(pitch_recall(self.y_pred, self.y_gt))
        self.metrics["pf"].append(pitch_f_measure(self.y_pred, self.y_gt))
        self.metrics["tp"].append(tab_precision(self.y_pred, self.y_gt))
        self.metrics["tr"].append(tab_recall(self.y_pred, self.y_gt))
        self.metrics["tf"].append(tab_f_measure(self.y_pred, self.y_gt))
        self.metrics["tdr"].append(tab_disamb(self.y_pred, self.y_gt))
        
    def save_results_csv(self):
        output = {}
        for key in self.metrics.keys():
            if key != "data":
                vals = self.metrics[key]
                mean = np.mean(vals)
                std = np.std(vals)
                output[key] = vals + [mean, std]
        output["data"] =  self.metrics["data"]
        df = pd.DataFrame.from_dict(output)
        df.to_csv(self.save_folder + "results.csv") 
        
##################################
########### EXPERIMENT ###########
##################################

tabcnn = TabCNN()

print("logging model...")
tabcnn.build_model()
tabcnn.log_model()

for fold in range(6):
    print("\nfold " + str(fold))
    tabcnn.partition_data(fold)
    print("building model...")
    tabcnn.build_model()
    print("training...")
    tabcnn.train()
    tabcnn.save_weights()
    print("testing...")
    tabcnn.test()
    tabcnn.save_predictions()
    print("evaluation...")
    tabcnn.evaluate()
print("saving results...")
tabcnn.save_results_csv()